<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_T10_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
def multiple_min_eigen(A,m,tol,max_iter,pr=False,v_0=[]):
    n=len(A)
    B=np.copy(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/np.sqrt(n))#Normalizado
    approximations=np.diag(B)
    approximations=np.sort(approximations)
    eigenvalues=np.zeros(m)
    eigenvectors=[]
    #print(approximations)
    L,U=factorizar_LU(A)
    for j in range(0,m):
        v_0=(np.zeros(n)+1/np.sqrt(n))#Normalizado
        alpha=approximations[j]
        A=shift(B,alpha)
        ##########
        v_1=solve(L,U,v_0)#Se resuelve el sistema
        lambda_0=np.dot(v_0,v_1)
        lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
        count=0
        while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
            count+=1
            v_0=v_1/np.linalg.norm(v_1)#Se normaliza
            v_1=solve(L_m=L,U_m=U,b=v_0)#iteracion
            lambda_0=lambda_1
            lambda_1=np.dot(v_0,v_1)
        eigenvalues[j]=(1/lambda_1)+alpha
        eigenvectors.append(v_1/np.linalg.norm(v_1))
        ###########
    return(eigenvalues,eigenvectors)

    import numpy as np
import math
def max_eigen(A,tol,max_iter,pr=True,v_0=[]):
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/math.sqrt(n))#Normalizado
    v_1=np.dot(A,v_0)#iteracion
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Normalizacion
        v_1=np.dot(A,v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>max_iter):
        return("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
        return([lambda_1,v_1])
def min_eigen(A,tol,max_iter,pr=True,v_0=[]):
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/math.sqrt(n))#Normalizado
    L,U=factorizar_LU(A)
    v_1=solve(L,U,v_0)#Se resuelve el sistema
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Se normaliza
        v_1=solve(L_m=L,U_m=U,b=v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>=max_iter):
        return("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
        return([1/lambda_1,v_1])
def deflacion(A,l,v):
    for i in range(len(A)):
        for j in range(len(A[0])):
            A[i][j]-=l*v[i]*v[j]
    return(A)
import numpy as np
def factorizar_LU(matriz):#Codigo reciclado: Factorizacion LU
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i][k]*U[k][j]
            L[i][j]=matriz[i][j]-sum
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j][k]*U[k][i]
                U[j][i]=(matriz[j][i]-sum)/L[j][j]
            else: 
                U[i][j]=1 
                if(L[i][i]==0):
                    return("Error en la construcción de L")
    return([L,U])

def superior(list,b):#Codigo reciclado:Resolver sistema triangular
  n=len(list)
  X=[b[n-1]/list[n-1][n-1]]
  X_inv=[X[0]]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[n-i-1][n-1-j]
    X.append((b[n-1-i]-acum)/list[n-1-i][n-1-i])
    X_inv.insert(0,X[i])
  return X_inv

#Codigo reciclado:Resolver sistema triangular
def inferior(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=[b[0]/list[0][0]]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i][j]
    X.append((b[i]-acum)/list[i][i])
  return X
def solve(L_m,U_m,b):
    y=inferior(L_m,b)#Resolver Ly=b
    return(superior(U_m,y))#Resolver Ux=y
def Jacobi_rot(A,tol,max_Iter):
    count=0
    n=len(A)
    P=np.identity(n)
    while((check_diag(A,tol)==False)&(count<=max_Iter)):
        p,q=entrada_max(A)
        a=A[p][q]
        if(A[p][p]==A[q][q]):
            theta=np.pi/4
        else:
            theta=0.5*np.arctan(2*a/(A[p][p]-A[q][q]))
        #Ya tenemos la rotacion y podemos hacer el producto de matrices
        R=np.identity(n)
        R[p][p]=np.cos(theta)
        R[p][q]=-np.sin(theta)
        R[q][p]=-R[p][q]
        R[q][q]=R[p][p]
        A=np.dot(R.T,np.dot(A,R))#Se puede mejorar, modificando solo aquellas entradas que cambian.
        count+=1
        P=np.dot(P,R)
    if(count>max_Iter):
        print("Maxima iteracion alcanzada")
    else:
        print("num. Iteraciones: ", str(count))
    return(P,np.diagonal(A))
:
def check_diag(A,tol,pr=False):#Pensado para matrices simetricas
    boolean=True
    breaker=False
    for i in range(len(A)):
        for j in range(i,len(A)):
            if(i!=j):
                if(abs(A[i][j])>tol):
                    boolean=False;
                    breaker=True;
                    break;
        if(breaker==True):
            break
    return(boolean)
def entrada_max(A):#Regresa una lista [i,j], tal que a_{i,j} es el valor maximo en valor absoluto.
#Pensado para matrices simetricas
    max=abs(A[0][1])
    index=[0,1]
    for i in range(len(A)):
        for j in range(i,len(A[0])):
            if(i!=j):
                if(max<abs(A[i][j])):
                    max=abs(A[i][j])
                    index=i,j
    return(index)
import numpy as np